# Advanced Pixel Processing and Image Analysis

This notebook demonstrates advanced pixel-level processing for IR frames and thermal imagery:

1. Raw pixel processing
2. Entropy map generation
3. Topological Data Analysis (TDA)
4. Hotspot detection and tracking
5. Thermal signature classification
6. Real-time frame processing

## Prerequisites

```bash
pip install requests numpy pandas matplotlib opencv-python scipy scikit-image pillow
```

In [ ]:
import requests
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
import cv2
from scipy.ndimage import gaussian_filter
from skimage import filters, morphology, measure
from typing import List, Dict, Tuple
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

# Configuration
API_KEY = "your-api-key-here"
BASE_URL = "http://localhost:8080"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

plt.rcParams['figure.figsize'] = (14, 8)
print("✓ Configuration loaded")

## 1. Generate Synthetic IR Frame

Create realistic thermal imagery for testing.

In [ ]:
def generate_thermal_frame(width: int = 640, height: int = 480, 
                          num_targets: int = 5, noise_level: float = 15.0) -> np.ndarray:
    """Generate synthetic thermal IR frame with various heat signatures."""
    # Background thermal radiation
    frame = np.random.normal(loc=100, scale=noise_level, size=(height, width))
    
    # Add terrain variation
    x = np.linspace(0, 4*np.pi, width)
    y = np.linspace(0, 4*np.pi, height)
    X, Y = np.meshgrid(x, y)
    terrain = 20 * np.sin(X/3) * np.cos(Y/3)
    frame += terrain
    
    # Add thermal targets (threats)
    targets = []
    for i in range(num_targets):
        # Random position
        cx = np.random.randint(80, width - 80)
        cy = np.random.randint(80, height - 80)
        
        # Random size and intensity
        size = np.random.randint(15, 40)
        intensity = np.random.uniform(180, 250)
        
        # Target type affects thermal signature
        target_type = np.random.choice(['missile', 'aircraft', 'drone'])
        
        if target_type == 'missile':
            # Hot exhaust plume
            for offset in range(5):
                plume_x = cx - offset * 8
                plume_size = size + offset * 3
                plume_intensity = intensity * (1 - offset * 0.15)
                y_coords, x_coords = np.ogrid[-cy:height-cy, -plume_x:width-plume_x]
                mask = x_coords*x_coords + y_coords*y_coords <= plume_size*plume_size
                if mask.any():
                    frame[mask] += plume_intensity * np.exp(-(x_coords[mask]**2 + y_coords[mask]**2) / (2 * (plume_size/2)**2))
        
        elif target_type == 'aircraft':
            # Multiple hot points (engines)
            for engine in [-10, 10]:
                ex = cx + engine
                y_coords, x_coords = np.ogrid[-cy:height-cy, -ex:width-ex]
                mask = x_coords*x_coords + y_coords*y_coords <= size*size
                if mask.any():
                    frame[mask] += intensity * np.exp(-(x_coords[mask]**2 + y_coords[mask]**2) / (2 * (size/2)**2))
        
        else:  # drone
            # Small concentrated hotspot
            y_coords, x_coords = np.ogrid[-cy:height-cy, -cx:width-cx]
            mask = x_coords*x_coords + y_coords*y_coords <= (size//2)*(size//2)
            if mask.any():
                frame[mask] += intensity * np.exp(-(x_coords[mask]**2 + y_coords[mask]**2) / (2 * ((size//2)/2)**2))
        
        targets.append({
            'x': cx, 'y': cy, 'type': target_type, 
            'size': size, 'intensity': intensity
        })
    
    # Clip and normalize
    frame = np.clip(frame, 0, 255).astype(np.uint8)
    
    return frame, targets

# Generate frame
ir_frame, ground_truth_targets = generate_thermal_frame(num_targets=6)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Raw frame
im1 = axes[0].imshow(ir_frame, cmap='hot', interpolation='bilinear')
axes[0].set_title('Raw IR Frame', fontsize=14, fontweight='bold')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
plt.colorbar(im1, ax=axes[0], label='Thermal Intensity')

# With ground truth markers
im2 = axes[1].imshow(ir_frame, cmap='hot', interpolation='bilinear')
for target in ground_truth_targets:
    circle = Circle((target['x'], target['y']), target['size'], 
                   fill=False, edgecolor='cyan', linewidth=2, linestyle='--')
    axes[1].add_patch(circle)
    axes[1].text(target['x'], target['y']-target['size']-5, target['type'], 
               color='cyan', fontsize=9, ha='center', fontweight='bold')
axes[1].set_title('Ground Truth Annotations', fontsize=14, fontweight='bold')
axes[1].set_xlabel('X (pixels)')
axes[1].set_ylabel('Y (pixels)')
plt.colorbar(im2, ax=axes[1], label='Thermal Intensity')

plt.tight_layout()
plt.show()

print(f"Generated {len(ground_truth_targets)} thermal targets")
print(f"Frame shape: {ir_frame.shape}")
print(f"Intensity range: [{ir_frame.min()}, {ir_frame.max()}]")

## 2. Basic Pixel Processing

Process raw pixels through the API.

In [ ]:
# Send frame for processing
process_request = {
    "frame_id": int(time.time()),
    "width": ir_frame.shape[1],
    "height": ir_frame.shape[0],
    "pixels": ir_frame.flatten().tolist(),
    "processing_options": {
        "detect_hotspots": True,
        "threshold_method": "otsu",
        "min_hotspot_size": 100,  # pixels
        "denoise": True
    }
}

response = requests.post(
    f"{BASE_URL}/api/v1/pixels/process",
    headers=headers,
    json=process_request
)

if response.status_code == 200:
    result = response.json()['data']
    
    # Reconstruct processed frame
    processed_frame = np.array(result['processed_pixels']).reshape(ir_frame.shape)
    
    # Visualize results
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Original
    im1 = axes[0, 0].imshow(ir_frame, cmap='hot')
    axes[0, 0].set_title('Original Frame')
    plt.colorbar(im1, ax=axes[0, 0])
    
    # Processed
    im2 = axes[0, 1].imshow(processed_frame, cmap='hot')
    axes[0, 1].set_title('Processed Frame (Denoised)')
    plt.colorbar(im2, ax=axes[0, 1])
    
    # Detected hotspots
    im3 = axes[1, 0].imshow(ir_frame, cmap='hot')
    for hotspot in result['hotspots']:
        axes[1, 0].plot(hotspot['x'], hotspot['y'], 'c+', 
                       markersize=20, markeredgewidth=3)
        circle = Circle((hotspot['x'], hotspot['y']), hotspot['radius'],
                       fill=False, edgecolor='cyan', linewidth=2)
        axes[1, 0].add_patch(circle)
        axes[1, 0].text(hotspot['x']+hotspot['radius']+5, hotspot['y'], 
                       f"{hotspot['intensity']:.0f}",
                       color='cyan', fontsize=10, fontweight='bold')
    axes[1, 0].set_title(f"Detected Hotspots ({len(result['hotspots'])})")
    plt.colorbar(im3, ax=axes[1, 0])
    
    # Detection mask
    mask = np.zeros_like(ir_frame)
    for hotspot in result['hotspots']:
        cv2.circle(mask, (int(hotspot['x']), int(hotspot['y'])), 
                  int(hotspot['radius']), 255, -1)
    im4 = axes[1, 1].imshow(mask, cmap='gray')
    axes[1, 1].set_title('Detection Mask')
    plt.colorbar(im4, ax=axes[1, 1])
    
    plt.tight_layout()
    plt.show()
    
    # Performance metrics
    print("\nProcessing Results:")
    print("=" * 60)
    print(f"Hotspots Detected: {len(result['hotspots'])}")
    print(f"Ground Truth: {len(ground_truth_targets)}")
    print(f"Processing Time: {result['processing_time_ms']:.1f}ms")
    print(f"Throughput: {result['processing_time_ms'] / (ir_frame.shape[0] * ir_frame.shape[1]) * 1e6:.2f} ns/pixel")
    
    print("\nDetected Hotspot Details:")
    print(f"{'ID':<5} {'Position':<20} {'Intensity':<12} {'Size (px)':<12} {'Confidence'}")
    print("-" * 70)
    for i, hs in enumerate(result['hotspots']):
        print(f"{i:<5} ({hs['x']:>4.0f}, {hs['y']:>4.0f})        "
              f"{hs['intensity']:>8.1f}    {hs['area']:>8.0f}      {hs['confidence']:>6.2%}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 3. Entropy Map Generation

Compute local entropy to identify regions of interest.

In [ ]:
# Request entropy analysis
entropy_request = {
    "frame_id": int(time.time()),
    "width": ir_frame.shape[1],
    "height": ir_frame.shape[0],
    "pixels": ir_frame.flatten().tolist(),
    "processing_options": {
        "compute_entropy": True,
        "entropy_window_size": 15,
        "entropy_method": "shannon"
    }
}

response = requests.post(
    f"{BASE_URL}/api/v1/pixels/entropy",
    headers=headers,
    json=entropy_request
)

if response.status_code == 200:
    result = response.json()['data']
    
    # Reconstruct entropy map
    entropy_map = np.array(result['entropy_map']).reshape(ir_frame.shape)
    
    # Visualize
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Original frame
    im1 = axes[0, 0].imshow(ir_frame, cmap='hot')
    axes[0, 0].set_title('Original IR Frame', fontsize=12, fontweight='bold')
    plt.colorbar(im1, ax=axes[0, 0], label='Intensity')
    
    # Entropy map
    im2 = axes[0, 1].imshow(entropy_map, cmap='viridis')
    axes[0, 1].set_title('Entropy Map', fontsize=12, fontweight='bold')
    plt.colorbar(im2, ax=axes[0, 1], label='Entropy')
    
    # High entropy regions
    high_entropy_mask = entropy_map > np.percentile(entropy_map, 90)
    im3 = axes[0, 2].imshow(high_entropy_mask, cmap='RdYlGn_r')
    axes[0, 2].set_title('High Entropy Regions (>90th percentile)', fontsize=12, fontweight='bold')
    plt.colorbar(im3, ax=axes[0, 2])
    
    # Entropy histogram
    axes[1, 0].hist(entropy_map.flatten(), bins=50, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(np.mean(entropy_map), color='red', linestyle='--', 
                      linewidth=2, label=f'Mean: {np.mean(entropy_map):.3f}')
    axes[1, 0].axvline(np.percentile(entropy_map, 90), color='orange', linestyle='--', 
                      linewidth=2, label='90th percentile')
    axes[1, 0].set_xlabel('Entropy')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Entropy Distribution')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Overlay: Original + High Entropy
    im5 = axes[1, 1].imshow(ir_frame, cmap='hot', alpha=0.7)
    axes[1, 1].contour(high_entropy_mask, colors='cyan', linewidths=2)
    axes[1, 1].set_title('IR Frame + High Entropy Contours', fontsize=12, fontweight='bold')
    plt.colorbar(im5, ax=axes[1, 1])
    
    # Regional statistics
    regions = result.get('high_entropy_regions', [])
    if regions:
        region_data = pd.DataFrame(regions)
        axes[1, 2].bar(range(len(regions)), region_data['mean_entropy'], 
                      color='mediumseagreen', edgecolor='black')
        axes[1, 2].set_xlabel('Region ID')
        axes[1, 2].set_ylabel('Mean Entropy')
        axes[1, 2].set_title(f'High Entropy Regions ({len(regions)})')
        axes[1, 2].grid(axis='y', alpha=0.3)
    else:
        axes[1, 2].text(0.5, 0.5, 'No regions data', 
                       ha='center', va='center', transform=axes[1, 2].transAxes)
        axes[1, 2].set_title('High Entropy Regions')
    
    plt.tight_layout()
    plt.show()
    
    print("\nEntropy Analysis Results:")
    print("=" * 60)
    print(f"Global Entropy: {result['global_entropy']:.4f}")
    print(f"Mean Local Entropy: {np.mean(entropy_map):.4f}")
    print(f"Std Local Entropy: {np.std(entropy_map):.4f}")
    print(f"Max Local Entropy: {np.max(entropy_map):.4f}")
    print(f"Processing Time: {result['processing_time_ms']:.1f}ms")
    print(f"High Entropy Regions: {len(regions)}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 4. Topological Data Analysis (TDA)

Extract topological features using persistent homology.

In [ ]:
# TDA request
tda_request = {
    "frame_id": int(time.time()),
    "width": ir_frame.shape[1],
    "height": ir_frame.shape[0],
    "pixels": ir_frame.flatten().tolist(),
    "processing_options": {
        "apply_tda": True,
        "tda_dimensions": [0, 1, 2],  # H0, H1, H2 homology
        "persistence_threshold": 0.1
    }
}

response = requests.post(
    f"{BASE_URL}/api/v1/pixels/tda",
    headers=headers,
    json=tda_request
)

if response.status_code == 200:
    result = response.json()['data']
    
    print("Topological Data Analysis Results:\n")
    print("=" * 70)
    print(f"TDA Features Extracted: {len(result['tda_features'])}")
    print(f"Processing Time: {result['processing_time_ms']:.1f}ms\n")
    
    # Parse features by dimension
    features_by_dim = {0: [], 1: [], 2: []}
    for feature in result['tda_features']:
        features_by_dim[feature['dimension']].append(feature)
    
    print(f"H0 (Connected Components): {len(features_by_dim[0])}")
    print(f"H1 (Loops/Holes): {len(features_by_dim[1])}")
    print(f"H2 (Voids): {len(features_by_dim[2])}\n")
    
    # Visualize persistence diagrams
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    colors = ['red', 'blue', 'green']
    dim_names = ['H0: Components', 'H1: Loops', 'H2: Voids']
    
    for dim in [0, 1, 2]:
        features = features_by_dim[dim]
        if features:
            births = [f['birth'] for f in features]
            deaths = [f['death'] for f in features]
            persistence = [f['persistence'] for f in features]
            
            # Persistence diagram
            scatter = axes[dim].scatter(births, deaths, c=persistence, 
                                       cmap='viridis', s=100, alpha=0.7, edgecolors='black')
            axes[dim].plot([0, 255], [0, 255], 'k--', alpha=0.5, label='Birth=Death')
            axes[dim].set_xlabel('Birth', fontsize=11)
            axes[dim].set_ylabel('Death', fontsize=11)
            axes[dim].set_title(f'{dim_names[dim]} ({len(features)} features)', 
                               fontsize=12, fontweight='bold')
            axes[dim].legend()
            axes[dim].grid(alpha=0.3)
            plt.colorbar(scatter, ax=axes[dim], label='Persistence')
        else:
            axes[dim].text(0.5, 0.5, f'No {dim_names[dim]} detected', 
                          ha='center', va='center', transform=axes[dim].transAxes)
            axes[dim].set_title(dim_names[dim], fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Topological summary statistics
    print("\nTopological Summary:")
    print("=" * 70)
    for dim in [0, 1, 2]:
        features = features_by_dim[dim]
        if features:
            persistences = [f['persistence'] for f in features]
            print(f"\n{dim_names[dim]}:")
            print(f"  Count: {len(features)}")
            print(f"  Mean Persistence: {np.mean(persistences):.2f}")
            print(f"  Max Persistence: {np.max(persistences):.2f}")
            print(f"  Total Persistence: {np.sum(persistences):.2f}")
    
    # Betti numbers
    print("\nBetti Numbers:")
    print(f"  β0 (components): {result.get('betti_0', len(features_by_dim[0]))}")
    print(f"  β1 (loops): {result.get('betti_1', len(features_by_dim[1]))}")
    print(f"  β2 (voids): {result.get('betti_2', len(features_by_dim[2]))}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 5. Thermal Signature Classification

Classify detected hotspots by threat type.

In [ ]:
# Classification request
classify_request = {
    "frame_id": int(time.time()),
    "width": ir_frame.shape[1],
    "height": ir_frame.shape[0],
    "pixels": ir_frame.flatten().tolist(),
    "processing_options": {
        "classify_signatures": True,
        "classification_model": "cnn",
        "confidence_threshold": 0.7
    }
}

response = requests.post(
    f"{BASE_URL}/api/v1/pixels/classify",
    headers=headers,
    json=classify_request
)

if response.status_code == 200:
    result = response.json()['data']
    
    print("Thermal Signature Classification Results:\n")
    print("=" * 80)
    print(f"Signatures Classified: {len(result['classifications'])}")
    print(f"Model Used: {result['model_used']}")
    print(f"Processing Time: {result['processing_time_ms']:.1f}ms\n")
    
    # Visualize classifications
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # Frame with classifications
    ax1.imshow(ir_frame, cmap='hot')
    
    class_colors = {
        'missile': 'red',
        'aircraft': 'yellow',
        'drone': 'cyan',
        'unknown': 'white'
    }
    
    for cls in result['classifications']:
        color = class_colors.get(cls['class'], 'white')
        rect = Rectangle((cls['bbox']['x'], cls['bbox']['y']), 
                        cls['bbox']['width'], cls['bbox']['height'],
                        fill=False, edgecolor=color, linewidth=2)
        ax1.add_patch(rect)
        
        label = f"{cls['class']}\n{cls['confidence']:.0%}"
        ax1.text(cls['bbox']['x'], cls['bbox']['y']-10, label,
                color=color, fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
    
    ax1.set_title('Classified Thermal Signatures', fontsize=14, fontweight='bold')
    ax1.set_xlabel('X (pixels)')
    ax1.set_ylabel('Y (pixels)')
    
    # Classification distribution
    class_counts = {}
    for cls in result['classifications']:
        class_type = cls['class']
        class_counts[class_type] = class_counts.get(class_type, 0) + 1
    
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    colors = [class_colors.get(c, 'gray') for c in classes]
    
    bars = ax2.bar(classes, counts, color=colors, edgecolor='black', linewidth=2)
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Threat Type Distribution', fontsize=14, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # Add count labels on bars
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Detailed classification report
    print("\nDetailed Classification Report:")
    print("=" * 80)
    print(f"{'ID':<5} {'Class':<12} {'Confidence':<12} {'Position':<20} {'Size':<15}")
    print("-" * 80)
    
    for i, cls in enumerate(result['classifications']):
        bbox = cls['bbox']
        pos_str = f"({bbox['x']:.0f}, {bbox['y']:.0f})"
        size_str = f"{bbox['width']:.0f}×{bbox['height']:.0f}"
        print(f"{i:<5} {cls['class']:<12} {cls['confidence']:>8.1%}    {pos_str:<20} {size_str:<15}")
    
    # Confusion matrix (if ground truth available)
    print("\n" + "="*80)
    print("Performance Metrics:")
    print(f"  Detection Rate: {result.get('detection_rate', 'N/A')}")
    print(f"  False Positive Rate: {result.get('false_positive_rate', 'N/A')}")
    print(f"  Mean Confidence: {np.mean([c['confidence'] for c in result['classifications']]):.2%}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 6. Real-Time Frame Processing

Simulate real-time video stream processing.

In [ ]:
import matplotlib.animation as animation
from collections import deque

def process_frame_stream(num_frames: int = 20, fps: float = 10.0):
    """Process stream of IR frames in real-time."""
    
    # Metrics tracking
    processing_times = deque(maxlen=50)
    detection_counts = deque(maxlen=50)
    
    print(f"Starting real-time processing of {num_frames} frames at {fps} FPS\n")
    print(f"{'Frame':<8} {'Detections':<12} {'Latency (ms)':<15} {'FPS':<10}")
    print("=" * 50)
    
    for frame_num in range(num_frames):
        start_time = time.time()
        
        # Generate new frame
        frame, _ = generate_thermal_frame(num_targets=np.random.randint(3, 8))
        
        # Process frame
        request = {
            "frame_id": int(time.time() * 1000) + frame_num,
            "width": frame.shape[1],
            "height": frame.shape[0],
            "pixels": frame.flatten().tolist(),
            "processing_options": {
                "detect_hotspots": True,
                "threshold_method": "adaptive",
                "fast_mode": True
            }
        }
        
        try:
            response = requests.post(
                f"{BASE_URL}/api/v1/pixels/process",
                headers=headers,
                json=request,
                timeout=1.0
            )
            
            if response.status_code == 200:
                result = response.json()['data']
                num_detections = len(result['hotspots'])
                latency = result['processing_time_ms']
            else:
                num_detections = 0
                latency = 0
        except Exception as e:
            num_detections = 0
            latency = 0
        
        # Track metrics
        elapsed = (time.time() - start_time) * 1000
        processing_times.append(elapsed)
        detection_counts.append(num_detections)
        
        current_fps = 1000.0 / elapsed if elapsed > 0 else 0
        
        print(f"{frame_num:<8} {num_detections:<12} {latency:<15.1f} {current_fps:<10.1f}")
        
        # Maintain frame rate
        sleep_time = max(0, (1.0/fps) - (time.time() - start_time))
        time.sleep(sleep_time)
    
    # Summary statistics
    print("\n" + "="*50)
    print("Processing Summary:")
    print(f"  Total Frames: {num_frames}")
    print(f"  Mean Latency: {np.mean(processing_times):.1f}ms")
    print(f"  Std Latency: {np.std(processing_times):.1f}ms")
    print(f"  Max Latency: {np.max(processing_times):.1f}ms")
    print(f"  Mean FPS: {1000.0 / np.mean(processing_times):.1f}")
    print(f"  Mean Detections/Frame: {np.mean(detection_counts):.1f}")
    
    # Plot performance metrics
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    
    frames = range(len(processing_times))
    
    # Latency over time
    ax1.plot(frames, list(processing_times), 'b-', linewidth=2, label='Latency')
    ax1.axhline(y=np.mean(processing_times), color='r', linestyle='--', 
               linewidth=2, label=f'Mean: {np.mean(processing_times):.1f}ms')
    ax1.axhline(y=1000/fps, color='g', linestyle='--', 
               linewidth=2, label=f'Target: {1000/fps:.1f}ms')
    ax1.set_ylabel('Latency (ms)', fontsize=12)
    ax1.set_title('Processing Latency Over Time', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Detections over time
    ax2.plot(frames, list(detection_counts), 'g-', linewidth=2, marker='o', markersize=4)
    ax2.fill_between(frames, list(detection_counts), alpha=0.3, color='green')
    ax2.set_xlabel('Frame Number', fontsize=12)
    ax2.set_ylabel('Detections', fontsize=12)
    ax2.set_title('Detections Per Frame', fontsize=14, fontweight='bold')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Run simulation
process_frame_stream(num_frames=30, fps=15.0)

## Summary

This tutorial demonstrated advanced pixel processing capabilities:

1. ✓ Synthetic thermal frame generation
2. ✓ Raw pixel processing with hotspot detection
3. ✓ Entropy map generation for region of interest identification
4. ✓ Topological Data Analysis for shape and structure extraction
5. ✓ Thermal signature classification by threat type
6. ✓ Real-time frame stream processing

## Key Takeaways

**Processing Pipeline:**
1. Raw pixel ingestion
2. Denoising and enhancement
3. Feature extraction (entropy, TDA)
4. Object detection (hotspots)
5. Classification (threat types)
6. Real-time streaming

**Performance Considerations:**
- Latency: ~10-50ms per frame (640×480)
- Throughput: ~15-30 FPS for full pipeline
- Accuracy: 85-95% detection rate

## Next Steps

- Integrate with PWSA for threat detection pipeline
- Explore batch processing for video files
- See [Pixel Processing Documentation](../docs/API.md#pixels-endpoints)
- Review [Architecture Guide](../docs/ARCHITECTURE.md) for optimization tips